In [1]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import cv2

In [3]:
interpreter = tf.lite.Interpreter(model_path='lite-model_movenet_singlepose_lightning_3.tflite')
interpreter.allocate_tensors()


In [4]:
def draw_keypoints(frame, keypoints, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > confidence_threshold:
            cv2.circle(frame, (int(kx), int(ky)), 4, (0,255,0), -1)

In [5]:
EDGES = {
    (0, 1): 'm',
    (0, 2): 'c',
    (1, 3): 'm',
    (2, 4): 'c',
    (0, 5): 'm',
    (0, 6): 'c',
    (5, 7): 'm',
    (7, 9): 'm',
    (6, 8): 'c',
    (8, 10): 'c',
    (5, 6): 'y',
    (5, 11): 'm',
    (6, 12): 'c',
    (11, 12): 'y',
    (11, 13): 'm',
    (13, 15): 'm',
    (12, 14): 'c',
    (14, 16): 'c'
}

In [7]:
def draw_connections(frame, keypoints, edges, confidence_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    
    for edge, color in edges.items():
        p1, p2 = edge
        y1, x1, c1 = shaped[p1]
        y2, x2, c2 = shaped[p2]
        
        if (c1 > confidence_threshold) & (c2 > confidence_threshold):      
            cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0,0,255), 2)

In [9]:
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    
    # Reshape image
    img = frame.copy()
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192,192)
    input_image = tf.cast(img, dtype=tf.float32)
    
    # Setup input and output 
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    # Make predictions 
    interpreter.set_tensor(input_details[0]['index'], np.array(input_image))
    interpreter.invoke()
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    print(keypoints_with_scores)
    
    # Rendering 
    draw_connections(frame, keypoints_with_scores, EDGES, 0.4)
    draw_keypoints(frame, keypoints_with_scores, 0.4)
    
    cv2.imshow('MoveNet Lightning', frame)
    
    if cv2.waitKey(10) & 0xFF==ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()

[[[[0.59762037 0.46200007 0.4815253 ]
   [0.5647888  0.47455978 0.55003977]
   [0.5701611  0.4193293  0.61603856]
   [0.5735327  0.47450697 0.3915458 ]
   [0.597036   0.34487426 0.7651329 ]
   [0.72797054 0.55008173 0.7170236 ]
   [0.7630822  0.2560525  0.74894726]
   [0.88147366 0.62112856 0.24567434]
   [0.93698025 0.20064381 0.04916706]
   [0.8823492  0.66634864 0.07132512]
   [0.9554943  0.34731182 0.01553182]
   [1.0062386  0.5370765  0.0521023 ]
   [1.0026127  0.32323796 0.01698634]
   [0.8591     0.7069041  0.01393486]
   [0.8653709  0.40687057 0.00979165]
   [0.8620535  0.7782463  0.02902852]
   [0.89645076 0.5857391  0.00382532]]]]
[[[[0.59870124 0.46439117 0.55381095]
   [0.5643488  0.47597227 0.5617968 ]
   [0.56946796 0.42073762 0.6466919 ]
   [0.57524866 0.46974844 0.49373135]
   [0.5973481  0.34191352 0.60138446]
   [0.7241421  0.55054224 0.5513389 ]
   [0.75551003 0.25174755 0.767439  ]
   [0.8840389  0.6207527  0.22493994]
   [0.9363606  0.1866025  0.0431921 ]
   [0.880